In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
from config import *
import random
from itertools import combinations,permutations
import pickle
import time

/home/jeff/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jeff/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jeff/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jeff/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWa

In [2]:
tf.reset_default_graph()
x = tf.placeholder(tf.float32, shape=[batch_size, N])
y = tf.placeholder(tf.float32, shape=[batch_size, K])
R_init = tf.placeholder(tf.float32, shape=[batch_size, N])

net_dict = {}
bp_iter_num = 5
RNN = 1
ss = np.hstack([0,np.arange(n)[::-1]+1])
int_L = 5
deci_L = 2
step = 2**-deci_L
_min = -2**(int_L+deci_L)*step
_max = (2**(int_L+deci_L)-1)*step
fix = 0
if(fix):
    inf_num = 2**(int_L-1)
else:
    inf_num = 1000

# initial
for i in range(n+1):
    for j in range(N):
        net_dict["L_{0}{1}{2}".format(i,j,0)] = tf.zeros((batch_size))
        net_dict["R_{0}{1}{2}".format(i,j,0)] = tf.zeros((batch_size))

if(RNN):
    LV = tf.Variable(np.float32(np.ones((n,N,1))))
    RV = tf.Variable(np.float32(np.ones((n,N,1))))
else:
    LV = tf.Variable(np.float32(np.ones((n,N,bp_iter_num))))
    RV = tf.Variable(np.float32(np.ones((n,N,bp_iter_num))))

for j in range(N):
    net_dict["L_{0}{1}{2}".format(n,j,0)] = tf.ones((1))*x[:,j]    
    net_dict["R_{0}{1}{2}".format(0,j,0)] = R_init[:,j]*inf_num

for i in range(n+1):
    for j in range(N):
        for k in range(bp_iter_num):
            net_dict["output_L_{0}{1}{2}".format(i,j,k)] = tf.zeros((batch_size))
            net_dict["output_R_{0}{1}{2}".format(i,j,k)] = tf.zeros((batch_size))
            
# bp algorithm
for k in range(bp_iter_num):
    if(RNN):
        itr = 0
    else:
        itr = k
    for i in range(n,0,-1):
        for phi in range(2**ss[i]):
            psi = int(np.floor(phi/2))
            if(np.mod(phi,2)!=0):
                for omega in range(2**(n-ss[i])):
                    net_dict["R_{0}{1}{2}".format(n+1-i,psi+2*omega*2**(ss[i]-1),0)] = RV[n-i,psi+2*omega*2**(ss[i]-1),itr]*fFunction(net_dict["L_{0}{1}{2}".format(n+1-i,psi+(2*omega+1)*2**(ss[i]-1),0)]+net_dict["R_{0}{1}{2}".format(n-i,psi+(2*omega+1)*2**(ss[i]-1),0)], net_dict["R_{0}{1}{2}".format(n-i,psi+2*omega*2**(ss[i]-1),0)])
                    net_dict["R_{0}{1}{2}".format(n+1-i,psi+(2*omega+1)*2**(ss[i]-1),0)] = net_dict["R_{0}{1}{2}".format(n-i,psi+(2*omega+1)*2**(ss[i]-1),0)]+RV[n-i,psi+(2*omega+1)*2**(ss[i]-1),itr]*fFunction(net_dict["L_{0}{1}{2}".format(n+1-i,psi+2*omega*2**(ss[i]-1),0)],net_dict["R_{0}{1}{2}".format(n-i,psi+2*omega*2**(ss[i]-1),0)])
                    
    for i in range(1,n+1):
        for phi in range(2**ss[i]):
            psi = int(np.floor(phi/2))
            if(np.mod(phi,2)!=0):
                for omega in range(2**(n-ss[i])):
                    net_dict["L_{0}{1}{2}".format(n-i,psi+2*omega*2**(ss[i]-1),0)] = LV[n-i,psi+2*omega*2**(ss[i]-1),itr]*fFunction(net_dict["L_{0}{1}{2}".format(n+1-i,psi+2*omega*2**(ss[i]-1),0)],net_dict["L_{0}{1}{2}".format(n+1-i,psi+(2*omega+1)*2**(ss[i]-1),0)]+net_dict["R_{0}{1}{2}".format(n-i,psi+(2*omega+1)*2**(ss[i]-1),0)])             
                    net_dict["L_{0}{1}{2}".format(n-i,psi+(2*omega+1)*2**(ss[i]-1),0)] = net_dict["L_{0}{1}{2}".format(n+1-i,psi+(2*omega+1)*2**(ss[i]-1),0)]+LV[n-i,psi+(2*omega+1)*2**(ss[i]-1),itr]*fFunction(net_dict["L_{0}{1}{2}".format(n+1-i,psi+2*omega*2**(ss[i]-1),0)],net_dict["R_{0}{1}{2}".format(n-i,psi+2*omega*2**(ss[i]-1),0)])

    for i in range(n+1):
        for j in range(N):
            net_dict["output_L_{0}{1}{2}".format(i,j,k)] = net_dict["L_{0}{1}{2}".format(i,j,0)]
            net_dict["output_R_{0}{1}{2}".format(i,j,k)] = net_dict["R_{0}{1}{2}".format(i,j,0)]
            
y_output = tf.zeros((1))
for i in range(N):
    if(FZlookup[i] == -1):
        y_output = tf.concat([y_output,net_dict["L_{0}{1}{2}".format(0,i,0)]],0)
y_output = tf.transpose(tf.reshape(y_output[1:],(K,batch_size)))*-1
loss = 1.0*tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=y_output,labels=y))
sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True))  #allow tensorflow to automatically allocate device
sess.run(tf.global_variables_initializer())

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [3]:
def dict2arr(_net_dict):
    output = np.zeros((batch_size, n+1, N, bp_iter_num, 2))
    for i in range(n+1):
        for j in range(N):
            for k in range(bp_iter_num):
                output[:,i,j,k,0] = _net_dict["output_L_{0}{1}{2}".format(i,j,k)]
                output[:,i,j,k,1] = _net_dict["output_R_{0}{1}{2}".format(i,j,k)]
    return output

In [4]:
def get_data_idx(err_map,indices):
    a,b = np.where(err_map == -1)
    i = 0
    y = []
    for c in np.unique(a):
        idx = np.where(a == c)[0]
        y.append(indices[b[idx]])
    return np.unique(a),y

In [5]:
tStart = time.time()
max_flip = 1

best_val_ber = 1
load_weight = 1
quantize_weight = 0 #0 for non-quantize, 1 for normal, 2 for binarized, 3 for bin, 4 for binarized bin
bin_bit = 3  # number of different value
binary_prec = 6 # binary precision, binary_prec must >= bin_bit

saver = tf.train.Saver()
if(load_weight):
    saver.restore(sess, 'Model/'+str(N)+'_0.0_5.0_5_1_0.ckpt')

test_nfails = np.zeros((len(ebn0)))
test_loss = np.zeros((len(ebn0)))
test_nframe = np.zeros((len(ebn0)))
BF_test_nfails = np.zeros(((len(ebn0)),max_flip))
BF_test_nframe = np.zeros(((len(ebn0)),max_flip))
y_pred_layer = np.zeros((batch_size,K,max_flip+1))
fail_frame_layer = np.zeros((batch_size,max_flip+1))
err_map = np.zeros((batch_size,K,max_flip+1))

wordRandom = np.random.RandomState(word_seed-200)
noiseRandom = np.random.RandomState(noise_seed-200)
zero = np.zeros(batch_size)
flip_bit_order = np.sort(indices[:K])

data_num = 3000

cnt_list = []
for j in range(len(ebn0)):
    X = []
    Y = []
    Z = []
    cnt = 0
    while(len(X)<data_num):
        print(cnt)
        cnt = cnt + 1
        x_test, y_test = gendata(j,True,True)
        r_layer = np.zeros((batch_size,N,max_flip+1))
        l = 0 # layer 0
        r_layer[:,FZlookup == 0,l] = 1
        y_pred_layer[:,:,0], _loss, _net_dict = sess.run(fetches=[y_output, loss, net_dict], feed_dict={x: x_test, y: y_test, R_init: r_layer[:,:,l]})
        test_loss[j] = _loss + test_loss[j]
        uhat = np.zeros((batch_size,K))
        uhat[y_pred_layer[:,:,0]>=0] = 1
        fail_frame_layer[:,0] = np.logical_or.reduce(uhat!=y_test,1)
        fail_frame_layer[:,1] = np.logical_or.reduce(uhat!=y_test,1)
        test_nfails[j] = test_nfails[j] + sum(sum(uhat!=y_test))        
        test_nframe[j] = test_nframe[j] + sum(fail_frame_layer[:,0])
        output = dict2arr(_net_dict)
        
        for k in range(K):
            l = 1 # layer 1
            r_layer[:,:,l] = r_layer[:,:,l-1].copy()
            flip_bit = flip_bit_order[k]
            idx = len(np.where(FZlookup[:flip_bit] == -1)[0]) # get the right idx of y_pred
            r_layer[:,flip_bit,l] = np.sign(y_pred_layer[:,idx,l-1])
            y_pred_layer[:,:,l], _loss = sess.run(fetches=[y_output, loss], feed_dict={x: x_test, y: y_test, R_init: r_layer[:,:,l]})
            uhat = np.zeros((batch_size,K))
            uhat[y_pred_layer[:,:,l]>=0] = 1
            fail_frame = np.logical_or.reduce(uhat!=y_test,1)
            fail_frame_layer[:,1] = fail_frame_layer[:,1] * fail_frame
            err_map[:,k,0] = fail_frame - fail_frame_layer[:,0] # find new corrected one
                    
        BF_test_nframe[j,0] = BF_test_nframe[j,0] + sum(fail_frame_layer[:,1])
        a, b = get_data_idx(err_map[:,:,0],flip_bit_order)
        X.extend(output[a,:,:,:])
        Y.extend(b)
        Z.extend(y_test[a,:])
    X = np.array(X)[:data_num]
    Y = Y[:data_num]
    Z = Z[:data_num]
    if(CRC):
        pickle.dump(X, open("./Data/CRC_X_SNR_{}_{}_1_0.p".format(N,int(ebn0[j])), "wb"))
        pickle.dump(Y, open("./Data/CRC_Y_SNR_{}_{}_1_0.p".format(N,int(ebn0[j])), "wb"))
    else:
        pickle.dump(X, open("./Data/X_SNR_{}_{}_1_0.p".format(N,int(ebn0[j])), "wb"))
        pickle.dump(Y, open("./Data/Y_SNR_{}_{}_1_0.p".format(N,int(ebn0[j])), "wb"))
        pickle.dump(Z, open("./Data/Z_SNR_{}_{}_1_0.p".format(N,int(ebn0[j])), "wb"))
    print('At SNR: {}, Generated number of data: {}'.format(int(ebn0[j]),len(Y)))
    cnt_list.append(cnt)
print('Test SNR:       ',ebn0)
print('Test Loss:      ',test_loss/cnt_list)
print('Test BER:       ',test_nfails/cnt_list/(batch_size*K))
print('Test FER:       ',test_nframe/cnt_list/(batch_size))
for i in range(max_flip):
    print('Test BF_{0} FER:   {1}'.format(int(i+1),BF_test_nframe[:,i]/cnt_list/(batch_size)))
tEnd = time.time()
print(tEnd-tStart)

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from Model/16_0.0_5.0_5_1_0.ckpt
0
1
At SNR: 0, Generated number of data: 3000
0
1
2
At SNR: 1, Generated number of data: 3000
0
1
2
3
At SNR: 2, Generated number of data: 3000
Test SNR:        [0. 1. 2.]
Test Loss:       [0.28986324 0.19284699 0.11169343]
Test BER:        [0.1264875  0.07963722 0.04304542]
Test FER:        [0.30168667 0.19290667 0.10596   ]
Test BF_1 FER:   [0.27389333 0.17466222 0.09563333]
22.870747566223145


In [6]:
print('Test SNR:       ',ebn0)
print('Test Loss:      ',test_loss/cnt_list)
print('Test BER:       ',test_nfails/cnt_list/(batch_size*K))
print('Test FER:       ',test_nframe/cnt_list/(batch_size))
for i in range(max_flip):
    print('Test BF_{0} FER:   {1}'.format(int(i+1),BF_test_nframe[:,i]/cnt_list/(batch_size)))
tEnd = time.time()

Test SNR:        [0. 1. 2.]
Test Loss:       [0.28986324 0.19284699 0.11169343]
Test BER:        [0.1264875  0.07963722 0.04304542]
Test FER:        [0.30168667 0.19290667 0.10596   ]
Test BF_1 FER:   [0.27389333 0.17466222 0.09563333]
